## 2.1 基础操作  
从接口的角度来讲，对tensor的操作可分为两类：
- torch.function，如torch.save等。
- tensor.function，如tensor.view等

从存储的角度来讲，对tensor的操作又可分为两类：
- 不会修改自身的数据，如 a.add(b)， 加法的结果会返回一个新的tensor。
- 会修改自身的数据，如 a.add_(b)， 加法的结果仍存储在a中，a被修改了     


**新建tensor**

|函数|功能|
|:---:|:---:|
|Tensor(\*sizes)|基础构造函数|
|tensor(data,)|类似np.array的构造函数|
|ones(\*sizes)|全1Tensor|
|zeros(\*sizes)|全0Tensor|
|eye(\*sizes)|对角线为1，其他为0|
|arange(s,e,step|从s到e，步长为step|
|linspace(s,e,steps)|从s到e，均匀切分成steps份|
|rand/randn(\*sizes)|均匀/标准分布|
|normal(mean,std)/uniform(from,to)|正态分布/均匀分布|
|randperm(m)|随机排列|

这些创建方法都可以在创建的时候指定数据类型dtype和存放device(cpu/gpu).


其中使用`Tensor`函数新建tensor是最复杂多变的方式，它既可以接收一个list，并根据list的数据新建tensor，也能根据指定的形状新建tensor，还能传入其他的tensor

In [0]:
import torch as t
import numpy as np

In [41]:
a = t.Tensor(3, 4)
a

tensor([[-1.1018e-21,  0.0000e+00,  4.4842e-44,  0.0000e+00],
        [        nan,  0.0000e+00,  2.1667e-04,  5.3599e-08],
        [ 1.3666e+22,  1.0978e-05,  1.0920e-05,  5.4720e+22]])

In [42]:
t.arange(0, 6, 1)

tensor([0, 1, 2, 3, 4, 5])

In [43]:
t.linspace(0, 6, 3)

tensor([0., 3., 6.])

In [44]:
t.randn(2, 3)

tensor([[-1.1860, -0.5908,  0.6800],
        [ 1.1175, -0.4167, -0.1546]])

In [45]:
t.randn(2, 3, device=t.device('cuda'))

tensor([[-1.0042,  2.7375, -0.4175],
        [ 1.1392,  0.2088,  0.9575]], device='cuda:0')

In [46]:
# 用list数据创建tensor
b = t.Tensor([[1, 2], [3, 4]])
b

tensor([[1., 2.],
        [3., 4.]])

In [47]:
b.tolist()

[[1.0, 2.0], [3.0, 4.0]]

In [48]:
size = b.size()
size

torch.Size([2, 2])

`tensor.size()`返回`torch.Size`对象，它是tuple的子类，但其使用方式与tuple略有区别

In [49]:
c = t.Tensor(size)
c

tensor([[-1.1017e-21,  0.0000e+00],
        [ 4.4842e-44,  0.0000e+00]])

要注意的是，`t.Tensor(*sizes)`创建tensor时，系统不会马上分配空间    
只是会计算剩余的内存是否足够使用，使用到tensor时才会分配，而其它操作都是在创建完tensor之后马上进行空间分配

In [50]:
# 还可利用tensor.shape直接查看tensor的形状，等价于tensor.size()
c.shape

torch.Size([2, 2])

### torch.tensor     
在0.4版本新增加的一个新版本的创建tensor方法，使用的方法，和参数几乎和`np.array`完全一致

In [51]:
# 标量
scalar = t.tensor(3.1415926)
scalar, scalar.shape

(tensor(3.1416), torch.Size([]))

In [52]:
# 向量
vector = t.tensor([1, 2, 3])
vector, vector.shape

(tensor([1, 2, 3]), torch.Size([3]))

In [53]:
# 矩阵
matrix = t.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
matrix, matrix.shape

(tensor([[1, 2, 3],
         [4, 5, 6],
         [7, 8, 9]]), torch.Size([3, 3]))

In [54]:
tensor = t.Tensor(1, 2)
tensor, tensor.shape

(tensor([[-4.4602e-21,  0.0000e+00]]), torch.Size([1, 2]))

#### Key : `torch.tensor` 与 `torch.Tensor `  
`torch.tensor`  
- 仅仅是python函数，会将data中数据进行拷贝(不是直接引用)，根据原生数据类型生成相应的    
  `torch.LongTensor`、`torch.FLoatTensor`、`torch.DoubleTensor`        
- 使用方法、参数和`np.array`很相近 


`torch.Tensor `   
- 是python类，是默认张量类型`torch.FloatTensor()`的别名
- 使用时，会调用Tensor类的构造函数`__init__`，生成单精度浮点类型的张量

## 2.2 常用Tensor操作    
（1）调整`Tensor`形状     
- `tensor.view`方法调整tensor的形状，须保证调整前后元素总数一致。`view`不会修改自身的数据，返回的新tensor与源tensor共享内存      
- resize    

（2）调整维度     
`squeeze`、`unsqueeze`函数

In [55]:
a =  t.arange(0, 6)
a.view(2, 3)

tensor([[0, 1, 2],
        [3, 4, 5]])

In [56]:
# 当某一维为-1时，会自动计算大小
b = a.view(-1, 2)
b

tensor([[0, 1],
        [2, 3],
        [4, 5]])

In [57]:
# 在第一维+1， 下标从0开始
c = b.unsqueeze(1)
# 3个1x2矩阵
c, c.shape

(tensor([[[0, 1]],
 
         [[2, 3]],
 
         [[4, 5]]]), torch.Size([3, 1, 2]))

In [58]:
d = b.unsqueeze(-2)
d, d.shape

(tensor([[[0, 1]],
 
         [[2, 3]],
 
         [[4, 5]]]), torch.Size([3, 1, 2]))

In [59]:
e = b.view(1, 1, 1, 2, 3)
e, e.shape

(tensor([[[[[0, 1, 2],
            [3, 4, 5]]]]]), torch.Size([1, 1, 1, 2, 3]))

In [60]:
# 压缩第0维
f = e.squeeze(0)
f, f.shape

(tensor([[[[0, 1, 2],
           [3, 4, 5]]]]), torch.Size([1, 1, 2, 3]))

In [61]:
# 把所有1 维度的压缩
g = e.squeeze()
g, g.shape

(tensor([[0, 1, 2],
         [3, 4, 5]]), torch.Size([2, 3]))

In [62]:
# a 值改变以后， b也会改变
a[0] = 111
b

tensor([[111,   1],
        [  2,   3],
        [  4,   5]])

#### `resize`       
是另一种可用来调整`size`的方法，它可以修改tensor的大小    
如果新size超过了原size，自动分配新的内存空间     
若新size小于原size，则之前的数据依旧会被保存

In [63]:
b = a.resize_(1, 3)
b

tensor([[111,   1,   2]])

In [64]:
b = a.resize_(3, 3)
b

tensor([[         111,            1,            2],
        [           3,            4,            5],
        [249108103217, 236223201333, 219043332154]])

## 2.3 索引操作    
Tensor支持与numpy.ndarray类似的索引操作，用法相似

In [65]:
a = t.randn(3, 4)
a 

tensor([[-0.3431, -2.1090, -0.0639, -1.7011],
        [ 0.0961, -0.5252, -1.7006,  1.2466],
        [ 0.4138, -0.0662, -0.7400,  1.1371]])

In [66]:
a[0]

tensor([-0.3431, -2.1090, -0.0639, -1.7011])

In [67]:
a[0, :]

tensor([-0.3431, -2.1090, -0.0639, -1.7011])

In [68]:
# 注意：操作相似，但形状不同
b = a[0:1, :2]
print(b, b.shape)

c = a[0, :2]
print(c, c.shape)

tensor([[-0.3431, -2.1090]]) torch.Size([1, 2])
tensor([-0.3431, -2.1090]) torch.Size([2])


In [69]:
# None类似于np.newaxis, 新增一个轴
a[None].shape

torch.Size([1, 3, 4])

In [70]:
a[:, None, :].shape

torch.Size([3, 1, 4])

In [71]:
# 等价于a.masked_select(a>1)
# 选择结果与原tensor不共享内存空间
a[a > -1]

tensor([-0.3431, -0.0639,  0.0961, -0.5252,  1.2466,  0.4138, -0.0662, -0.7400,
         1.1371])

### 常用的选择函数     
函数|功能|
:---:|:---:|
index_select(input, dim, index)|在指定维度dim上选取，比如选取某些行、某些列
masked_select(input, mask)|例子如上，a[a>0]，使用ByteTensor进行选取
non_zero(input)|非0元素的下标
gather(input, dim, index)|根据index，在dim维度上选取数据，**输出的size与index一样**      
  
  
 (1) **gather-聚集操作**     
 `gather`是一个比较复杂的操作，对一个2维tensor，输出的每个元素如下：

```python
out[i][j] = input[index[i][j]][j]  # dim=0
out[i][j] = input[i][index[i][j]]  # dim=1
```

In [72]:
a = t.arange(0, 16).view(4, 4)
a

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11],
        [12, 13, 14, 15]])

In [73]:
# 选取对角线元素  
index = t.LongTensor([[0, 1, 2, 3]])
a.gather(0, index)

tensor([[ 0,  5, 10, 15]])

In [74]:
# 选取反对角线元素
index = t.LongTensor([[3, 2, 1, 0]])

print(a.gather(1, index.t()))
print(a.gather(0, index))

tensor([[ 3],
        [ 6],
        [ 9],
        [12]])
tensor([[12,  9,  6,  3]])


In [75]:
# 选取双对角线元素   
index = t.LongTensor([[0, 1, 2, 3], [3, 2, 1, 0]]).t()

b = a.gather(1, index)
b

tensor([[ 0,  3],
        [ 5,  6],
        [10,  9],
        [15, 12]])

(2) **scatter-分散操作**    
与`gather`相对应的逆操作是`scatter_`，`gather`把数据从input中按index取出，而`scatter_`是把取出的数据再放回去。即`scatter_`函数是inplace操作

In [76]:
a = t.zeros(2, 4)
index = t.tensor([[2], [3]])
print(index[0][0])
print(index.shape)

a.scatter_(1, index, 1.23)

tensor(2)
torch.Size([2, 1])


tensor([[0.0000, 0.0000, 1.2300, 0.0000],
        [0.0000, 0.0000, 0.0000, 1.2300]])

### 高级索引    
0.2版本中完善了索引操作，目前已经支持绝大多数numpy的高级索引。   
高级索引可以看成是普通索引操作的扩展，但是高级索引操作的结果一般不和原始的Tensor共享内存

In [77]:
a = t.arange(0, 27).view(3, 3, 3)
a

tensor([[[ 0,  1,  2],
         [ 3,  4,  5],
         [ 6,  7,  8]],

        [[ 9, 10, 11],
         [12, 13, 14],
         [15, 16, 17]],

        [[18, 19, 20],
         [21, 22, 23],
         [24, 25, 26]]])

In [78]:
# 纯整数组索引--在每个维上指定任意索引
#  n, dim, index
a[[1, 2], [1, 2], [2, 0]] # 等价于 a[1,1,2], a[2,2,0]

tensor([14, 24])

In [79]:
# 支持广播，重复
a[[0, 1, 2], [0], [1]] # 等价于a[0,0,1] a[1,0,1] a[2,0,1]

tensor([ 1, 10, 19])

##  2.4 Other    

### GPU/CPU
tensor可以很随意的在gpu/cpu上传输。使用`tensor.cuda(device_id)`或者`tensor.cpu()`。另外一个更通用的方法是`tensor.to(device)`

In [86]:
a = t.randn(3, 4)
a, a.device

(tensor([[ 1.6830, -0.8484, -0.2639, -0.2613],
         [ 0.6256, -1.0901, -0.2841,  0.0257],
         [ 2.5757,  1.5335, -0.2268, -0.5638]]), device(type='cpu'))

In [88]:
device = t.device('cuda:0')
a.to(device)

tensor([[ 1.6830, -0.8484, -0.2639, -0.2613],
        [ 0.6256, -1.0901, -0.2841,  0.0257],
        [ 2.5757,  1.5335, -0.2268, -0.5638]], device='cuda:0')

**注意**
- 尽量使用`tensor.to(device)`, 将`device`设为一个可配置的参数，这样可以很轻松的使程序同时兼容GPU和CPU
- 数据在GPU之中传输的速度要远快于内存(CPU)到显存(GPU), 所以尽量避免频繁的在内存和显存中传输数据

### 持久化
Tensor的保存和加载十分的简单，使用`t.save和t.load`即可完成相应的功能      
在save/load时可指定使用的`pickle`模块，在load时还可将GPU tensor映射到CPU或其它GPU上

In [92]:
if t.cuda.is_available():
    # a 转为GPU0上的 tensor
    a = a.cuda(0)
    t.save(a, 'a.pth')

    # 加载为b， 存储于 GPU0上
    b = t.load('a.pth')
    # 存储于 cpu上
    c = t.load('a.pth', map_location=lambda storage, loc : storage)

a, b, c

(tensor([[ 1.6830, -0.8484, -0.2639, -0.2613],
         [ 0.6256, -1.0901, -0.2841,  0.0257],
         [ 2.5757,  1.5335, -0.2268, -0.5638]], device='cuda:0'),
 tensor([[ 1.6830, -0.8484, -0.2639, -0.2613],
         [ 0.6256, -1.0901, -0.2841,  0.0257],
         [ 2.5757,  1.5335, -0.2268, -0.5638]], device='cuda:0'),
 tensor([[ 1.6830, -0.8484, -0.2639, -0.2613],
         [ 0.6256, -1.0901, -0.2841,  0.0257],
         [ 2.5757,  1.5335, -0.2268, -0.5638]]))

### 向量化    
向量化计算是一种特殊的并行计算方式，相对于一般程序在同一时间只执行一个操作的方式，它可在同一时间执行多个操作，通常是对不同的数据执行同样的一个或一批指令，或者说把指令应用于一个数组/向量上   

向量化可极大提高科学运算的效率，Python本身是一门高级语言，使用很方便，但这也意味着很多操作很低效，尤其是`for`循环。在科学计算程序中应当极力避免使用Python原生的`for循环` 

In [0]:
def for_loop_add(x, y):
    result = []
    for i, j in zip(x, y):
        result.append(i + j)
    return t.tensor(result)

In [94]:
x = t.ones(100)
y = t.zeros(100)

%timeit -n 100 for_loop_add(x, y)
%timeit -n 100 x + y

100 loops, best of 3: 644 µs per loop
100 loops, best of 3: 3.16 µs per loop


二者有超过几十倍的速度差距，因此在实际使用中应尽量调用**内建函数(buildin-function)**，这些函数底层由C/C++实现，能通过执行底层优化实现高效计算     

因此在平时写代码时，就应养成向量化的思维习惯，千万避免对较大的tensor进行逐元素遍历

### 其他注意点：    

- 大多数`t.function`都有一个参数`out`，这时候产生的结果将保存在out指定tensor之中。
- `t.set_num_threads`可以设置PyTorch进行CPU多线程并行计算时候所占用的线程数，这个可以用来限制PyTorch所占用的CPU数目。
- `t.set_printoptions`可以用来设置打印tensor时的数值精度和格式

In [95]:
a = t.randn(2, 3)
a

tensor([[-0.3536, -1.2710,  0.9475],
        [-0.8725,  1.0671,  0.1073]])

In [96]:
t.set_printoptions(precision=12)
a

tensor([[-0.353581517935, -1.270960450172,  0.947500765324],
        [-0.872466742992,  1.067063808441,  0.107277430594]])